In [1]:
#Add Dependencies
import pandas as pd
import numpy as np
# import psycopg2 as pg
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
read_file = "Working/movies_metadata.csv"
write_file = "Working/movies_clean.csv"

#read_file = "/Users/archieanand/Desktop/Analysis/Working/movies_metadata.csv"
#Write_file = "/Users/archieanand/Desktop/Analysis/Working/movies_clean.csv"

In [3]:
# Import our input dataset
movies_df = pd.read_csv(read_file , low_memory=False)


In [4]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
movies_df = movies_df.drop(movies_df[movies_df.adult == "True"].index)

In [7]:
movies_df = movies_df.drop(movies_df[movies_df.status != "Released"].index)

In [8]:
movies_df= movies_df.drop(columns=["adult", "status", "homepage", "belongs_to_collection", "id","original_language", "overview", "poster_path", "spoken_languages", "tagline", "video"    ])
movies_df.head()

,budget,genres,imdb_id,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,vote_count
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",tt0114709,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",tt0113497,Jumanji,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",tt0113228,Grumpier Old Men,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0114885,Waiting to Exhale,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0
4,0,"[{'id': 35, 'name': 'Comedy'}]",tt0113041,Father of the Bride Part II,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0


In [9]:
eval(movies_df.at[0, 'production_companies'])[0]['name']

'Pixar Animation Studios'

In [10]:
for index, row in movies_df.iterrows():    
    try:        
        companies = movies_df.loc[index, 'production_companies']        
        company = eval(companies)[0]['name']    
    except:        
        company = "Other"    
    movies_df.at[index, 'production'] = company

In [11]:
for index, row in movies_df.iterrows():    
    try:        
        countries = movies_df.loc[index, 'production_countries']        
        country = eval(countries)[0]['name']    
    except:        
        country = "Other"    
    movies_df.at[index, 'country'] = country

In [12]:
for index, row in movies_df.iterrows():    
    try:        
        genres = movies_df.loc[index, 'genres']        
        genre = eval(genres)[0]['name']    
    except:        
        genres = "Other"    
    movies_df.at[index, 'new_genre'] = genre

In [13]:
movies_df['release_date']= pd.to_datetime(movies_df['release_date'])

In [14]:
for index, row in movies_df.iterrows():
    try:
        year = df1['release_date'].str[:2]
    except:     # if release_date is empty
        year = "other"
        newDate = "other"
    movies_df.at[index, 'year'] = year
    

In [21]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44678 entries, 0 to 45465
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   budget          44678 non-null  int64         
 1   imdb_id         44678 non-null  object        
 2   original_title  44678 non-null  object        
 3   popularity      44678 non-null  object        
 4   release_date    44678 non-null  datetime64[ns]
 5   revenue         44678 non-null  float64       
 6   runtime         44678 non-null  float64       
 7   title           44678 non-null  object        
 8   vote_average    44678 non-null  float64       
 9   vote_count      44678 non-null  float64       
 10  production      44678 non-null  object        
 11  country         44678 non-null  object        
 12  new_genre       44678 non-null  object        
 13  year            44678 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(8)


In [15]:
movies_df

,budget,genres,imdb_id,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,vote_count,production,country,new_genre,year
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",tt0114709,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,other
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",tt0113497,Jumanji,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,TriStar Pictures,United States of America,Adventure,other
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",tt0113228,Grumpier Old Men,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0,Warner Bros.,United States of America,Romance,other
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0114885,Waiting to Exhale,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0,Twentieth Century Fox Film Corporation,United States of America,Comedy,other
4,0,"[{'id': 35, 'name': 'Comedy'}]",tt0113041,Father of the Bride Part II,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,Sandollar Productions,United States of America,Comedy,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",tt6209470,رگ خواب,0.072051,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaT,0.0,90.0,Subdue,4.0,1.0,Other,Iran,Drama,other
45462,0,"[{'id': 18, 'name': 'Drama'}]",tt2028550,Siglo ng Pagluluwal,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,Century of Birthing,9.0,3.0,Sine Olivia,Philippines,Drama,other
45463,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",tt0303758,Betrayal,0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,Betrayal,3.8,6.0,American World Pictures,United States of America,Action,other
45464,0,[],tt0008536,Satana likuyushchiy,0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,Satan Triumphant,0.0,0.0,Yermoliev,Russia,Action,other


In [16]:
movies_df= movies_df.drop(columns=["genres", "production_companies", "production_countries"])
movies_df.head()

,budget,imdb_id,original_title,popularity,release_date,revenue,runtime,title,vote_average,vote_count,production,country,new_genre,year
0,30000000,tt0114709,Toy Story,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,other
1,65000000,tt0113497,Jumanji,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,TriStar Pictures,United States of America,Adventure,other
2,0,tt0113228,Grumpier Old Men,11.7129,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0,Warner Bros.,United States of America,Romance,other
3,16000000,tt0114885,Waiting to Exhale,3.859495,1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0,Twentieth Century Fox Film Corporation,United States of America,Comedy,other
4,0,tt0113041,Father of the Bride Part II,8.387519,1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,Sandollar Productions,United States of America,Comedy,other


In [17]:
movies_df.set_index("imdb_id")

,budget,original_title,popularity,release_date,revenue,runtime,title,vote_average,vote_count,production,country,new_genre,year
imdb_id,,,,,,,,,,,,,
tt0114709,30000000,Toy Story,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,other
tt0113497,65000000,Jumanji,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,TriStar Pictures,United States of America,Adventure,other
tt0113228,0,Grumpier Old Men,11.7129,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0,Warner Bros.,United States of America,Romance,other
tt0114885,16000000,Waiting to Exhale,3.859495,1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0,Twentieth Century Fox Film Corporation,United States of America,Comedy,other
tt0113041,0,Father of the Bride Part II,8.387519,1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,Sandollar Productions,United States of America,Comedy,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt6209470,0,رگ خواب,0.072051,NaT,0.0,90.0,Subdue,4.0,1.0,Other,Iran,Drama,other
tt2028550,0,Siglo ng Pagluluwal,0.178241,2011-11-17,0.0,360.0,Century of Birthing,9.0,3.0,Sine Olivia,Philippines,Drama,other
tt0303758,0,Betrayal,0.903007,2003-08-01,0.0,90.0,Betrayal,3.8,6.0,American World Pictures,United States of America,Action,other


In [18]:
movies_df['budget']=movies_df.budget.astype('int64')

In [19]:
movies_df = movies_df.dropna(axis = 0,how= 'any')

In [20]:
movies_df.to_csv(write_file)